# Análise dos Deputados Eleitos em 2022

Passada as eleições de primeiro turno que definiram a composição da câmara, tem-se a oportunidade de verificar padrões das candidaturas eleitas e característica da representatividade dos deputados eleitos. 

Esta análise se utilizou dos dados das candidaturas coletados e abertos pelo TSE e compilados pela plataforma basedosdados.org, que disponibiliza os dados para consulta através de banco de dados disponíveis para a consulta através do Google Big Query. Para filtrar os candidatos eleitos foi feito um scrapping dos dados das candidaturas eleitas do portal UOL. Este processo foi feito com uso de Python e os dados obtidos foram armazenados em um banco de dados no Google Big Query.

## Web Scrapping Deputados Eleitos

In [1]:
from unidecode import unidecode # Retirar acentos e caracteres especiais das strings
import pandas as pd # Manipulação dos dados
import pandas_gbq # Integração com Google Big Query
import requests # Requisição HTTP
from bs4 import BeautifulSoup  # Tratamento do HTML
from urllib.error import URLError, HTTPError # Detecção de possíveis erros na URL ou no arquivo HTML
from google.cloud import bigquery # Conexão com Big Query
from google.oauth2 import service_account # Conexão com Big Query

Utilizando a biblioteca requests para a requisição HTML e extração do conteúdo da página

In [2]:
url_dep = 'https://noticias.uol.com.br/eleicoes/2022/10/03/deputados-federais-eleitos-brasil-2022.htm'
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}

try:
    req_dep = requests.get(url_dep, headers=headers)
    html_dep = req_dep.content

except HTTPError as e:
    print(e.status, e.reason)

except URLError as e:
    print(e.reason)

soup_dep = BeautifulSoup(html_dep)
print(soup_dep.prettify())

Manipulando o HTML utilizando a biblioteca Beautiful Soup para localizar e extrair as informações dos candidatos eleitos.

Juntando as informações dos deputados eleitos com os respectivos estados através da criação de um dicionário, que é utilizado para a criação de um Data Frame do Pandas.

In [55]:
lista_estados = ['Acre', 'Alagoas', 'Amapá', 'Amazonas', 'Bahia', 'Ceará', 'Distrito Federal', 'Espírito Santo', 'Goiás', 'Maranhão', 'Mato Grosso', 'Mato Grosso do Sul', 'Minas Gerais', 'Pará', 'Paraíba',  'Paraná', 'Pernambuco', "Piauí", 'Rio de Janeiro', 'Rio Grande do Norte', 'Rio Grande do Sul', 'Rondônia', 'Roraima', 'Santa Catarina', 'São Paulo',  'Sergipe', 'Tocantins']
deputados = soup_dep.find_all("ul")[5:32]

dic_deputados = {}

for i in range(len(deputados)):
    for j in range(len(deputados[i].find_all('li'))):
        dic_deputados[deputados[i].find_all("li")[j].text] = lista_estados[i]

df_deputados = pd.DataFrame.from_dict(data=dic_deputados, orient='index').reset_index()
df_deputados.columns = ['candidatos', 'estado']
df_deputados

,candidatos,estado
0,"Socorro Neri (PP) - 25.842 votos - 5,95%",Acre
1,"Meire Serafim (UNIÃO) - 21.285 votos - 4,90%",Acre
2,"Coronel Ulysses (UNIÃO) - 21.075 votos - 4,85%",Acre
3,"Zezinho Barbary (PP) - 19.958 votos - 4,60%",Acre
4,"Gerlen Diniz (PP) - 19.560 votos - 4,50%",Acre
...,...,...
506,Alexandre Guimarães (REPUBLICANOS) - 54.703 vo...,Tocantins
507,"Carlos Gaguim (UNIÃO) - 52.203 votos - 6,29%",Tocantins
508,Ricardo Ayres (REPUBLICANOS) - 45.880 votos - ...,Tocantins
509,"Filipe Martins (PL) - 36.293 votos - 4,37%",Tocantins


Inserindo candidatos que estavam faltando nas informações do UOL.

In [56]:
df_deputados.loc[32] = ["Fausto Santos Jr (UNIÃO) - 87.876 votos", "Amazonas"]
df_deputados.loc[512] = ["Albuquerque (REPUBLICANOS) - 45.491 votos", "Roraima"]
df_deputados.loc[511] = ["Lazaro Botelho (PP) - 13.668 votos", "Tocantins"]

Criando função para a extração da informação de votos das strings obtidas do processo de scrapping.

In [57]:
def votos(x):
    if ":" in x:
        return (x.split(":")[1].split("-")[0])
    else:
        if x == "Paulo Abi-Ackel (PSDB) - 105.383 votos - 0,94%":
            return(x.split("-")[2])
        else:
            return(x.split("-")[1])

Aplicando transformações nas strings para a criação das colunas "nome",  "partido" e "votos".

In [58]:
df_deputados["nome"] = df_deputados.candidatos.apply(lambda x: x.split("(")[0].strip())
df_deputados['partido'] = df_deputados.candidatos.apply(lambda x: x.rsplit(")")[0].split("(")[-1])
df_deputados["votos"] = df_deputados.candidatos.apply(votos)

df_deputados["partido"] = df_deputados.partido.apply(lambda x: x.replace("Tio Rafa", "MDB"))
df_deputados["partido"] = df_deputados.partido.apply(lambda x: x.replace("União Brasil", "UNIÃO"))
df_deputados["partido"] = df_deputados.partido.apply(lambda x: x.replace("CID", "CIDADANIA") if len(x)==3 else x)
df_deputados["partido"] = df_deputados.partido.apply(lambda x: x.replace("PCdoB", "PC do B"))

df_deputados["votos"] = df_deputados.votos.apply(lambda x: x.replace(".", ""))
df_deputados["votos"] = df_deputados.votos.apply(lambda x: x.replace("votos", "")).astype(int)

df_deputados["nome"] = df_deputados.nome.apply(lambda x : x.replace("é Haroldo Cathedral", "Zé Haroldo Cathedral"))
df_deputados["nome"] = df_deputados.nome.apply(lambda x : x.replace("5,38% ", ""))
df_deputados["nome"] = df_deputados.nome.apply(lambda x : x.replace("'", ""))
df_deputados["nome"] = df_deputados.nome.apply(lambda x : x.replace("Jr.", "Jr"))
df_deputados["nome"] = df_deputados.nome.apply(lambda x : x.replace("Rafael Brito", "Rafael Brito (Tio Rafa)"))


df_deputados.drop(columns=["candidatos"], inplace=True)

df_deputados

,estado,nome,partido,votos
0,Acre,Socorro Neri,PP,25842
1,Acre,Meire Serafim,UNIÃO,21285
2,Acre,Coronel Ulysses,UNIÃO,21075
3,Acre,Zezinho Barbary,PP,19958
4,Acre,Gerlen Diniz,PP,19560
...,...,...,...,...
508,Tocantins,Ricardo Ayres,REPUBLICANOS,45880
509,Tocantins,Filipe Martins,PL,36293
510,Tocantins,Eli Borges,PL,35171
512,Roraima,Albuquerque,REPUBLICANOS,45491


Fazendo a conexão com o Google Big Query e enviando as informações do Data Frame Pandas para o banco de dados.

In [59]:
key_path = "/work/disco-plane-350516-7f9b3f5834fa.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])

df_deputados.to_gbq(credentials=credentials, destination_table='CongressoEleitos2022.DeputadosFederais', if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 13486.51it/s]


## Análise Exploratória dos dados

Através de consultas SQL aos bancos de dados do basedosdados.org foi feita uma análise exploratória dos dados na intenção de gerar insights sobre o o tamanho das bancadas associadas aos candidatos a presidência que disputarão o segundo turno, o financiamento das candidaturas, patrimônio declarado dos candidatos eleitos e recorte de gênero.

As informações foram cruzadas através de concatenação das strings de nome e sigla do partido. Apenas uma chave estava duplicada e foi removida através do filtro do sequencial do candidato conforme consulta abaixo.

In [60]:


df_1 = _deepnote_execute_sql("""SELECT
  cam.estado,
  cand.sigla_uf,
  cand.nome_urna,
  cand.sigla_partido,
  cand.nome,
  cand.sequencial
FROM
  `disco-plane-350516.CongressoEleitos2022.DeputadosFederais` cam
LEFT OUTER JOIN
  `basedosdados.br_tse_eleicoes.candidatos` cand
ON
  CONCAT(LOWER(cam.nome),cam.partido) = CONCAT(TRIM(LOWER(cand.nome_urna)),cand.sigla_partido)
WHERE
  cand.ano = 2022
  AND cand.cargo = "deputado federal"
  AND cand.sequencial NOT IN ("260001667973")
ORDER BY
  nome_urna;
""", 'SQL_12BCD23B_4F34_42B5_8311_851684283E9B')
df_1

,estado,sigla_uf,nome_urna,sigla_partido,nome,sequencial
0,Mato Grosso,MT,Abílio,PL,Abilio Jacques Brunini Moumer,110001612761
1,Amapá,AP,Acácio Favacho,MDB,Acácio Da Silva Favacho Neto,30001605451
2,Amazonas,AM,Adail Filho,REPUBLICANOS,Adail José Figueiredo Pinheiro,40001609376
3,Bahia,BA,Adolfo Viana,PSDB,Adolfo Viana De Castro Neto,50001613575
4,São Paulo,SP,Adriana Ventura,NOVO,Adriana Miguel Ventura,250001597671
...,...,...,...,...,...,...
508,Roraima,RR,Zé Haroldo Cathedral,PSD,José Haroldo Figueiredo Campos,230001600149
509,Bahia,BA,Zé Neto,PT,José Cerqueira De Santana Neto,50001605365
510,Minas Gerais,MG,Zé Silva,SOLIDARIEDADE,José Silva Soares,130001612991
511,Santa Catarina,SC,Zé Trovão,PL,Marcos Antonio Pereira Gomes,240001610976


Através da reportagem sobre as bancadas do congresso disponível no G1, estabeleceu-se três categorias de bancadas. Os que apoiam o candidato Lula, os que apoiam Bolsonaro e os partidos que não participam de nenhuma coligação.

In [61]:


df_bancadas = _deepnote_execute_sql("""SELECT
  CASE
    WHEN cand.sigla_partido IN ("PL","REPUBLICANOS","PP","PTB","PSC") THEN "Bolsonaro"
    WHEN cand.sigla_partido IN ("PT","PC do B","PV","PDT","PSOL","REDE","PSB","AVANTE","CIDADANIA","SOLIDARIEDADE","PROS") THEN "Lula"
  ELSE "Não Coligados"
END
  AS bancadas,
  COUNT(*) AS total_deputados,
  CONCAT(FORMAT("%2.2f",(COUNT(*)/513 * 100)), "%") AS percentual
FROM
  `disco-plane-350516.CongressoEleitos2022.DeputadosFederais` cam
LEFT OUTER JOIN
  `basedosdados.br_tse_eleicoes.candidatos` cand
ON
  CONCAT(LOWER(cam.nome),cam.partido) = CONCAT(TRIM(LOWER(cand.nome_urna)),cand.sigla_partido)
WHERE
  cand.ano = 2022
  AND cand.cargo = "deputado federal"
  AND cand.sequencial NOT IN ("260001667973")
GROUP BY
  bancadas
""", 'SQL_12BCD23B_4F34_42B5_8311_851684283E9B')
df_bancadas

,bancadas,total_deputados,percentual
0,Bolsonaro,194,37.82%
1,Não Coligados,175,34.11%
2,Lula,144,28.07%


In [62]:
_deepnote_run_altair(df_bancadas, """{"data":{"name":"placeholder"},"mark":{"type":"bar","tooltip":true},"height":220,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"bin":false,"axis":{"grid":false},"sort":{"order":"descending","encoding":"y"},"type":"nominal","field":"bancadas","scale":{"type":"linear","zero":false}},"y":{"axis":{"grid":false},"sort":null,"type":"quantitative","field":"total_deputados","scale":{"type":"linear","zero":true}},"color":{"sort":null,"type":"nominal","field":"bancadas","scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

Observa-se que existe uma proporção próxima de um terço para cada uma dos grupos, tendo a bancada do atual presidente uma considerável vantagem em relação aos números de seu concorrente. Na prática, quem for eleito terá de negociar com uma parcela significativa da câmara para poder aprovar projetos de lei e propostas de emenda à constituição de seu interesse. 

In [63]:


df_receita = _deepnote_execute_sql("""SELECT
  tb_eleitos.bancadas,
  tb_eleitos.sigla_partido,
  tb_eleitos.total_deputados,
  FORMAT("%'2.2f",tb_fin.receita_partido) AS receita_partido,
  CONCAT(FORMAT("%2.2f",tb_fin.receita_partido/ (
      SELECT
        SUM(valor_receita) AS receita_partido
      FROM
        `basedosdados.br_tse_eleicoes.receitas_candidato`
      WHERE
        ano=2022) * 100), "%") AS percentual_receita,
  FORMAT("%'2.2f",(tb_fin.receita_partido/tb_eleitos.total_deputados)) AS receita_por_deputado,
  tb_fin.receita_partido AS receita_partido_num
FROM (
  SELECT
    CASE
      WHEN cand.sigla_partido IN ("PL","REPUBLICANOS","PP","PTB","PSC") THEN "Bolsonaro"
      WHEN cand.sigla_partido IN ("PT","PC do B","PV","PDT","PSOL","REDE","PSB","AVANTE","CIDADANIA","SOLIDARIEDADE","PROS") THEN "Lula"
    ELSE "Não Coligados"
  END
    AS bancadas,
    cand.sigla_partido,
    COUNT(*) AS total_deputados,
  FROM
    `disco-plane-350516.CongressoEleitos2022.DeputadosFederais` cam
  LEFT OUTER JOIN
    `basedosdados.br_tse_eleicoes.candidatos` cand
  ON
    CONCAT(LOWER(cam.nome),cam.partido) = CONCAT(TRIM(LOWER(cand.nome_urna)),cand.sigla_partido)
  WHERE
    cand.ano = 2022
    AND cand.cargo = "deputado federal"
    AND cand.sequencial NOT IN ("260001667973")
  GROUP BY
    cand.sigla_partido) tb_eleitos
LEFT JOIN (
  SELECT
    sigla_partido,
    SUM(valor_receita) AS receita_partido
  FROM
    `basedosdados.br_tse_eleicoes.receitas_candidato`
  WHERE
    ano=2022
  GROUP BY
    sigla_partido) tb_fin
ON
  tb_eleitos.sigla_partido = tb_fin.sigla_partido
GROUP BY
  tb_eleitos.bancadas,
  tb_eleitos.sigla_partido,
  tb_eleitos.total_deputados,
  tb_fin.receita_partido
ORDER BY
  tb_fin.receita_partido DESC
""", 'SQL_12BCD23B_4F34_42B5_8311_851684283E9B')
df_receita

,bancadas,sigla_partido,total_deputados,receita_partido,percentual_receita,receita_por_deputado,receita_partido_num
0,Não Coligados,UNIÃO,59,"918,982,256.91",15.13%,"15,575,970.46",9.189823e+08
1,Lula,PT,68,"589,584,233.81",9.70%,"8,670,356.38",5.895842e+08
2,Bolsonaro,PL,99,"522,861,618.35",8.61%,"5,281,430.49",5.228616e+08
3,Bolsonaro,PP,47,"488,029,837.56",8.03%,"10,383,613.57",4.880298e+08
4,Não Coligados,MDB,42,"463,031,043.06",7.62%,"11,024,548.64",4.630310e+08
5,Não Coligados,PSD,42,"433,489,191.45",7.14%,"10,321,171.22",4.334892e+08
6,Bolsonaro,REPUBLICANOS,41,"371,638,899.61",6.12%,"9,064,363.41",3.716389e+08
7,Não Coligados,PSDB,13,"340,937,872.70",5.61%,"26,225,990.21",3.409379e+08
8,Lula,PSB,14,"305,571,150.87",5.03%,"21,826,510.78",3.055712e+08
9,Lula,PDT,17,"257,561,650.02",4.24%,"15,150,685.30",2.575617e+08


Na consulta acima temos o total de deputados eleitos por partido e a receita total destes partidos para o uso em campanha. 

Observamos uma correlação entre a receita de campanha com o número de deputados eleitos, quanto maior a receita do partido, mais deputados são eleitos. Porém, quando observamos a relação de receita por deputado eleito temos algumas distorções. O PL, que teve o maior número de deputados eleitos, teve o menor gasto por candidato eleito, em compensação o PTB, que teve somente um deputado eleito, gastou mais de vinte e uma vezes o que gastou o PL por deputado, sendo de longe o partido que mais gastou nessa relação.

In [64]:
_deepnote_run_altair(df_receita, """{"data":{"name":"placeholder"},"mark":{"type":"bar","tooltip":true},"height":220,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"sort":null,"type":"nominal","field":"sigla_partido","scale":{"type":"linear","zero":false}},"y":{"bin":false,"sort":null,"type":"quantitative","field":"receita_partido_num","scale":{"type":"linear","zero":true}},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false},"legend":null}}}""")

alt.Chart(...)

In [65]:


df_receita_bancadas = _deepnote_execute_sql("""SELECT
  tb_eleitos.bancadas,
  FORMAT("%'2.2f",SUM(tb_fin.receita_partido)) AS receita_por_bancada,
  CONCAT(FORMAT("%2.2f",SUM(tb_fin.receita_partido)/ (
      SELECT
        SUM(valor_receita)
      FROM
        `basedosdados.br_tse_eleicoes.receitas_candidato`
      WHERE
        ano=2022) * 100), "%") AS percentual_receita,
  SUM(tb_eleitos.deputados_eleitos) AS deputados_eleitos,
  CONCAT(FORMAT("%2.2f",(SUM(tb_eleitos.deputados_eleitos)/513 * 100)), "%") AS percentual_eleito,
  SUM(tb_fin.receita_partido) AS receita_por_bancada_num
FROM (
  SELECT
    sigla_partido,
    COUNT(*) AS deputados_eleitos,
    CASE
      WHEN cand.sigla_partido IN ("PL","REPUBLICANOS","PP","PTB","PSC") THEN "Bolsonaro"
      WHEN cand.sigla_partido IN ("PT","PC do B","PV","PDT","PSOL","REDE","PSB","AVANTE","CIDADANIA","SOLIDARIEDADE","PROS") THEN "Lula"
    ELSE "Não Coligados"
  END
    AS bancadas
  FROM
    `disco-plane-350516.CongressoEleitos2022.DeputadosFederais` cam
  LEFT OUTER JOIN
    `basedosdados.br_tse_eleicoes.candidatos` cand
  ON
    CONCAT(LOWER(cam.nome),cam.partido) = CONCAT(TRIM(LOWER(cand.nome_urna)),cand.sigla_partido)
  WHERE
    cand.ano = 2022
    AND cand.cargo = "deputado federal"
    AND cand.sequencial NOT IN ("260001667973")
  GROUP BY
    sigla_partido) tb_eleitos
LEFT JOIN (
  SELECT
    sigla_partido,
    SUM(valor_receita) AS receita_partido
  FROM
    `basedosdados.br_tse_eleicoes.receitas_candidato`
  WHERE
    ano=2022
  GROUP BY
    sigla_partido) tb_fin
ON
  tb_fin.sigla_partido = tb_eleitos.sigla_partido
GROUP BY
  tb_eleitos.bancadas
ORDER BY
  tb_eleitos.bancadas;
""", 'SQL_12BCD23B_4F34_42B5_8311_851684283E9B')
df_receita_bancadas

,bancadas,receita_por_bancada,percentual_receita,deputados_eleitos,percentual_eleito,receita_por_bancada_num
0,Bolsonaro,"1,596,111,355.01",26.27%,194,37.82%,1.596111e+09
1,Lula,"1,848,162,733.01",30.42%,144,28.07%,1.848163e+09
2,Não Coligados,"2,550,680,073.47",41.98%,175,34.11%,2.550680e+09


Conforme a consulta acima observamos que os partidos não coligados tiveram a maior parte da receita para campanha, o que não se traduziu no maior número de deputados eleitos. A bancada de apoio ao presidente Bolsonaro foi a mais eficiente no uso dos recursos, elegendo o maior percentual de deputados com a menor quantidade de recursos. A utilização da máquina pública pode explicar esta maior eficiência pois que muitos votos ainda tem uma relação clientelista com o uso dos recursos orçamentários dos deputados eleitos, o que é conhecido hoje como orçamento secreto.

In [66]:
_deepnote_run_altair(df_receita_bancadas, """{"data":{"name":"placeholder"},"mark":{"type":"bar","tooltip":true},"height":220,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"axis":{"grid":false},"sort":{"order":"descending","encoding":"y"},"type":"nominal","field":"bancadas","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"receita_por_bancada_num","scale":{"type":"linear","zero":true}},"color":{"sort":null,"type":"nominal","field":"bancadas","scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [67]:


df_4 = _deepnote_execute_sql("""SELECT
  tb_eleitos.bancadas,
  COUNT(*) AS deputados_eleitos,
  CONCAT(FORMAT("%2.2f",(COUNT(*)/513 * 100)), "%") AS percentual_eleito,
  FORMAT("%'2.2f", AVG(tb_patri.patrimonio_candidato)) AS patrimonio_medio,
  FORMAT("%'.2f",APPROX_QUANTILES(tb_patri.patrimonio_candidato,2)[OFFSET(0)]) AS minimo_patrimonio,
  FORMAT("%'.2f",APPROX_QUANTILES(tb_patri.patrimonio_candidato,2)[OFFSET(1)]) AS mediana_patrimonio,
  FORMAT("%'.2f",APPROX_QUANTILES(tb_patri.patrimonio_candidato,2)[OFFSET(2)]) AS maxima_patrimonio,
FROM (
  SELECT *,
    CASE WHEN cand.sigla_partido IN ("PL", "REPUBLICANOS", "PP", "PTB", "PSC") THEN "Bolsonaro"
    WHEN cand.sigla_partido IN ("PT","PC do B", "PV", "PDT", "PSOL","REDE", "PSB", "AVANTE", "CIDADANIA","SOLIDARIEDADE", "PROS") THEN "Lula"
    ELSE "Não Coligados"
    END AS bancadas
  FROM
    `disco-plane-350516.CongressoEleitos2022.DeputadosFederais` cam
  LEFT OUTER JOIN
    `basedosdados.br_tse_eleicoes.candidatos` cand
  ON
    CONCAT(LOWER(cam.nome),cam.partido) = CONCAT(TRIM(LOWER(cand.nome_urna)),cand.sigla_partido)
  WHERE
    cand.ano = 2022
    AND cand.cargo = "deputado federal"
    AND cand.sequencial NOT IN ("260001667973")) tb_eleitos
LEFT JOIN (
  SELECT
    sequencial_candidato, SUM(valor_item) AS patrimonio_candidato
  FROM
    `basedosdados.br_tse_eleicoes.bens_candidato`
  WHERE
    ano=2022
  GROUP BY sequencial_candidato) tb_patri
ON
  REPLACE(FORMAT('%12s',tb_patri.sequencial_candidato), " ", "0") = tb_eleitos.sequencial
GROUP BY
  tb_eleitos.bancadas
ORDER BY
  tb_eleitos.bancadas;
""", 'SQL_12BCD23B_4F34_42B5_8311_851684283E9B')
df_4

,bancadas,deputados_eleitos,percentual_eleito,patrimonio_medio,minimo_patrimonio,mediana_patrimonio,maxima_patrimonio
0,Bolsonaro,194,37.82%,"2,120,977.44","3,340.59","1,045,154.69","45,531,715.86"
1,Lula,144,28.07%,"2,772,398.06","3,573.12","623,279.79","107,545,780.55"
2,Não Coligados,175,34.11%,"3,698,833.28","18,327.66","1,393,523.71","65,903,045.68"


Na consulta acima temos as receitas segregadas por bancadas. Observa-se que o patrimônio médio não traduz bem a realidade da maioria dos deputados eleitos, pois a distribuição de patrimônio é muito desigual. Sendo assim, a mediana diz mais sobre a condição dos deputados eleitos, com destaque para a diferença entre a mediana dos patrimônios da bancada de Lula e dos partidos sem coligação, quase o dobro.

A diferença entre o patrimônio máximo e a mediana é gigante em todas bancadas ultrapassando em dois casos mais de 100 vezes. Esta característica de valores muito distantes da média faz com que o valor médio não traduza a realidade da maioria dos deputados eleitos.

In [68]:


df_5 = _deepnote_execute_sql("""SELECT
  tb_eleitos.genero,
  FORMAT("%'2.2f",SUM(tb_patri.patrimonio_candidato)) AS patrimonio_por_genero,
  COUNT(*) AS deputados_eleitos,
  CONCAT(FORMAT("%2.2f",(COUNT(*)/513 * 100)), "%") AS percentual_eleito,
  FORMAT("%'2.2f", AVG(tb_patri.patrimonio_candidato)) AS media_patrimonio,
  FORMAT("%'.2f",APPROX_QUANTILES(tb_patri.patrimonio_candidato,2)[OFFSET(0)]) AS minimo_patrimonio,
  FORMAT("%'.2f",APPROX_QUANTILES(tb_patri.patrimonio_candidato,2)[OFFSET(1)]) AS mediana_patrimonio,
  FORMAT("%'.2f",APPROX_QUANTILES(tb_patri.patrimonio_candidato,2)[OFFSET(2)]) AS maxima_patrimonio,
  FORMAT("%'.2f",APPROX_QUANTILES(tb_patri.patrimonio_candidato,10)[OFFSET(9)]) AS noventa_porcento_patrimonio,
FROM (
  SELECT *
  FROM
    `disco-plane-350516.CongressoEleitos2022.DeputadosFederais` cam
  LEFT OUTER JOIN
    `basedosdados.br_tse_eleicoes.candidatos` cand
  ON
    CONCAT(LOWER(cam.nome),cam.partido) = CONCAT(TRIM(LOWER(cand.nome_urna)),cand.sigla_partido)
  WHERE
    cand.ano = 2022
    AND cand.cargo = "deputado federal"
    AND cand.sequencial NOT IN ("260001667973")) tb_eleitos
LEFT JOIN (
  SELECT
    sequencial_candidato, SUM(valor_item) AS patrimonio_candidato
  FROM
    `basedosdados.br_tse_eleicoes.bens_candidato`
  WHERE
    ano=2022
  GROUP BY sequencial_candidato) tb_patri
ON
  REPLACE(FORMAT('%12s',tb_patri.sequencial_candidato), " ", "0") = tb_eleitos.sequencial
GROUP BY
  tb_eleitos.genero
ORDER BY
  tb_eleitos.genero
""", 'SQL_12BCD23B_4F34_42B5_8311_851684283E9B')
df_5

,genero,patrimonio_por_genero,deputados_eleitos,percentual_eleito,media_patrimonio,minimo_patrimonio,mediana_patrimonio,maxima_patrimonio,noventa_porcento_patrimonio
0,feminino,"90,026,822.45",91,17.74%,"1,343,683.92","3,340.59","496,712.54","15,762,410.22","2,525,000.00"
1,masculino,"978,208,721.29",422,82.26%,"3,176,002.34","4,000.00","1,167,092.40","107,545,780.55","4,797,415.75"


A participação feminina ainda é muito pequena na câmara dos deputados, não chegando a um quinto das candidaturas eleitas. Quando observamos o patrimônio dos candidatos eleitos segregados por gênero temos uma ideia da desigualdade entre os gêneros com 50% das mulheres eleitas com um patrimônio de menos de quinhentos mil reais, enquanto o patrimônio mediano para o gênero masculino é de mais de um milhão de reais.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=40e553a8-385c-41d9-81e6-7f2054338b1c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>